In this notebook, we explore the baseline measures for navground simulations to see how well these differenciate between behaviors. 
These measures are the safety margin violations, collisions, agent movement efficacy and deadlocks.

We focus on the cross scenario.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import scipy.spatial.distance as dist
from navground import sim, core

from tslearn.generators import random_walks
from tslearn import metrics
import seaborn as sns
import gudhi
from gudhi.wasserstein import wasserstein_distance
from navground.sim.ui.video import display_video_from_run, record_video_from_run
from functools import partial
from IPython.display import Image

# Create folder to save bottleneck distances between matching diagrams
import os
os.makedirs("Baseline_cross_matrices", exist_ok=True)
os.makedirs("plots", exist_ok=True)

In [ ]:
from perdiver.navground_io import parser

args = parser.parse_args([
        '--scenario', 'Cross',
        '--side', '10.0',
        '--num_runs', '12',
        '--num_steps', '600',
        '--time_step', '0.1',
        '--num_agents', '10',
        '--max_speed', '1.66',
        '--optimal_speed_min', '0.9',
        '--optimal_speed_max', '1.5',
        '--radius', '0.4',
        '--safety_margin', '0.1',
        '--behavior', 'HL',
        '--max_edge_length', '33.0',
        '--time_delay', '5',
        '--embedding_length', '11',
        '--epsilon', '50',
    ])

## 1. HL behavior

We run the experiment with HL behavior.

In [ ]:
from perdiver.navground_io import run_navground
runs = run_navground(args)

In [ ]:
display_video_from_run(run=runs[1], factor=3.0, fps=30)

In [ ]:
runs[0].collisions

In [ ]:
runs[0].safety_violations[145]

In [ ]:
runs[0].safety_violations.shape

In [ ]:
safety_violations_HL = []
for run_key in runs.keys():
    run = runs[run_key]
    safety_violations_HL.append(np.sum(run.safety_violations, axis=1))
# end for
safety_violations_HL = np.array(safety_violations_HL)

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))
signal = safety_violations_HL[1]
time = range(len(signal))
ax.plot(time, signal)
ax.set_ylim([0,1])

## 2. ORCA behavior

We run the experiment with ORCA behavior.

In [ ]:
args.behavior = "ORCA"
runs = run_navground(args)

In [ ]:
safety_violations_ORCA = []
for run_key in runs.keys():
    run = runs[run_key]
    safety_violations_ORCA.append(np.sum(run.safety_violations, axis=1))
# end for
safety_violations_ORCA = np.array(safety_violations_ORCA)

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))
signal = safety_violations_ORCA[6]
time = range(len(signal))
ax.plot(time, signal)
ax.set_ylim([0,1])

## 3. SocialForce behavior

We run the experiment with SF behavior.

In [ ]:
args.behavior = "SocialForce"
runs = run_navground(args)

In [ ]:
safety_violations_SF = []
for run_key in runs.keys():
    run = runs[run_key]
    safety_violations_SF.append(np.sum(run.safety_violations, axis=1))
# end for
safety_violations_SF = np.array(safety_violations_SF)

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))
signal = safety_violations_SF[1]
time = range(len(signal))
ax.plot(time, signal)
ax.set_ylim([0,1])

In [ ]:
all_safety_violations = np.vstack([
    safety_violations_HL, safety_violations_ORCA, safety_violations_SF
])
all_safety_violations.shape

In [ ]:
from sklearn.decomposition import PCA

fig, ax = plt.subplots(figsize=(5,5))

titles = ["safety violations"]

pca = PCA(n_components=2)
embedding = pca.fit_transform(all_safety_violations)
embedding_dict = {}
behavior_list = ["HL", "ORCA", "SF"]
for i, behavior in enumerate(behavior_list):
    embedding_dict[behavior] = embedding[args.num_runs*i:args.num_runs*(i+1)]
# end for

color_behavior = { "HL": "blue", "ORCA": "red", "SF": "green"}
for behavior in behavior_list:
    ax.scatter(embedding_dict[behavior][:,0], embedding_dict[behavior][:,1], color=color_behavior[behavior], label=behavior)

ax.set_title("safety violations")
plt.legend()
plt.savefig(os.path.join("plots", "PCA-projection-baseline.png"))